In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [5]:
import os
import sys

sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from mle_toolbox import load_result_logs
from mle_toolbox.utils import load_meta_log

# Load logs and create submission

In [7]:
experiment_dir = "../experiments/mlp_alexnet_bo_20"
meta_log, hyper_log = load_result_logs(experiment_dir)

meta_log = load_meta_log("../experiments/mlp_vgg_bo_50/meta_log.hdf5")

In [11]:
meta_log['b_1_eval_0'].meta.experiment_dir

'experiments/mlp_vgg_bo_30/21-05-31_b_1_eval_0/'

# Get submission ready based on `experiment_dir`

In [25]:
track = "mini_track"
if track == 'full_track': ROIs = ['WB']
else: ROIs = ['LOC','FFA','STS','EBA','PPA','V1','V2','V3','V4']

all_subjects = ['sub01', 'sub02', 'sub03', 'sub04', 'sub05',
                'sub06', 'sub07', 'sub08', 'sub09', 'sub10']

# Load in individual test npy files (subject/roi)
results = {}
for ROI in ROIs:
    ROI_results = {}
    for sub in all_subjects:
        hyper_sub = hyper_log[hyper_log.subject_id == sub]
        hyper_sub_roi = hyper_sub[hyper_sub.roi_type == ROI]
        # experiment_dir = hyper_sub_roi.experiment_dir.iloc[0]
        eval_id = hyper_sub_roi.run_id.iloc[0]
        experiment_dir = meta_log[eval_id].meta.experiment_dir
        mypath = os.path.join("../" + experiment_dir, "extra")
        try:
            sub_roi_layer = [f for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]
            ROI_result = np.load(mypath + "/" + sub_roi_layer[0])
            ROI_results[sub] = ROI_result
        except:
            print(mypath)
            continue
    results[ROI] = ROI_results

In [26]:
results

{'LOC': {'sub01': array([[-0.26316494, -0.26243958, -0.2920494 , ..., -0.10295969,
          -0.01802597,  0.0392699 ],
         [ 0.10421777, -0.02912289,  0.1737662 , ...,  0.16285208,
           0.21687338, -0.1623795 ],
         [-0.27902776, -0.2629026 , -0.53952324, ..., -0.1229055 ,
           0.08064914, -0.08407961],
         ...,
         [ 0.09071955,  0.14089239,  0.0682954 , ...,  0.07976509,
          -0.00949965,  0.03103483],
         [-0.18376495, -0.18006146, -0.36723283, ..., -0.12231883,
          -0.13455388,  0.06143023],
         [-0.07487796, -0.09191842, -0.2268797 , ..., -0.06249486,
          -0.01249808, -0.0264936 ]], dtype=float32),
  'sub02': array([[-0.08780873, -0.18012223, -0.10373879, ..., -0.21664378,
           0.14682107, -0.06195557],
         [ 0.34737045,  0.07668129,  0.10669487, ...,  0.1578801 ,
          -0.05562259,  0.01169663],
         [-0.07502262, -0.12452047, -0.10281879, ..., -0.193495  ,
           0.12622052, -0.02676702],
        

In [27]:
# Create model_submission sub directory
model_name = "2021-06-02_mlp_vgg_bo_50"
sub_dir = f"../data/submissions/{model_name}/{track}/"
if not os.path.exists(sub_dir):
    os.makedirs(sub_dir)
    
from utils.helper import save_dict
save_dict(results, sub_dir + track + ".pkl")

In [28]:
import zipfile

def zip(src, dst):
    zf = zipfile.ZipFile(dst, "w", zipfile.ZIP_DEFLATED)
    abs_src = os.path.abspath(src)
    for dirname, subdirs, files in os.walk(src):
        for filename in files:
            absname = os.path.abspath(os.path.join(dirname, filename))
            arcname = absname[len(abs_src) + 1:]
            if arcname in ["mini_track.pkl", "full_track.pkl"]:
                zf.write(absname, arcname)
    zf.close()

zip(sub_dir, sub_dir + track + ".zip")

# Manual Training of Final Predictor Networks

In [ ]:
# from utils.helper import get_encoding_data
# from encoding_models import EncoderFitter
# import shutil

# config_keys = ['dropout', 'hidden_act', 'learning_rate', 'num_hidden_layers',
#                'num_hidden_units', 'optimizer_class', 'w_decay']

# for e_id in meta_log.eval_ids:
#     best_id = np.argmax(meta_log[e_id].stats.best_bo_score.mean)
#     best_config = {}
#     for c_key in config_keys:
#         if type(meta_log[e_id].stats[c_key]) == list:
#             config_value = meta_log[e_id].stats[c_key][0][best_id].decode()
#         else:
#             config_value = meta_log[e_id].stats[c_key].mean[best_id]
#         best_config[c_key] = config_value
#     best_layer = meta_log[e_id].stats['layer_id'][0][best_id].decode()
#     #print(model_config, layer_id)
    
#     subject_id = hyper_log[hyper_log.run_id == e_id].subject_id.iloc[0]
#     roi_type = hyper_log[hyper_log.run_id == e_id].roi_type.iloc[0]
    
#     X, y, X_test = get_encoding_data(fmri_dir="../data/participants_data_v2021",
#                                      activations_dir='../data/features/alexnet/pca_100',
#                                      layer_id=best_layer,
#                                      subject_id=subject_id,
#                                      roi_type=roi_type)
#     fitter = EncoderFitter("mlp_network", 10,
#                            X, y, X_test)
#     model_params, y_pred = fitter.predict_on_test(best_config)
#     dirpath = "../" + meta_log[e_id].meta.experiment_dir + "extra/"
#     save_fname = dirpath + subject_id + "_" + roi_type + "_" + best_layer + "_test.npy"
#     if os.path.exists(dirpath) and os.path.isdir(dirpath):
#         shutil.rmtree(dirpath)
#     os.makedirs(dirpath)
#     np.save(save_fname , y_pred)
#     print(e_id, save_fname)